In [1]:
pip install osmnx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.2/100.2 kB 2.5 MB/s eta 0:00:00


In [2]:
import osmnx as ox
import networkx as nx
import matplotlib.pyplot as plt
from shapely.geometry import Point
import geopandas as gpd
import pandas as pd

In [11]:
pip install twilio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 17.4 MB/s eta 0:00:00


In [49]:
pip install openrouteservice

In [60]:
# --- New Help Station List with Phone Numbers ---
help_stations_with_phone = [
    {"coordinates": (30.3179409, 78.0500178), "phone_number": "+917617641909"},
    {"coordinates": (30.319395, 78.0503182), "phone_number": "+919389969916"},
    {"coordinates": (30.3250628, 78.0540411), "phone_number": "+917617641909"},
    {"coordinates": (30.3121892, 78.0474883), "phone_number": "+919389969916"},
    {"coordinates": (30.3257766, 78.0428731), "phone_number": "+917617641909"},
    {"coordinates": (30.3301055, 78.0503148), "phone_number": "+919389969916"},
    {"coordinates": (30.3196005, 78.0422984), "phone_number": "+917617641909"},
    {"coordinates": (30.3184253, 78.0424823), "phone_number": "+919389969916"},
    {"coordinates": (30.3207776, 78.0576667), "phone_number": "+917617641909"},
    {"coordinates": (30.3189236, 78.0506355), "phone_number": "+919389969916"},
    {"coordinates": (30.327541, 78.0479398), "phone_number": "+917617641909"},
    {"coordinates": (30.3163536, 78.054412), "phone_number": "+919389969916"},
    {"coordinates": (30.318361, 78.0527356), "phone_number": "+917617641909"}
]

# --- Unload Coordinates ---
help_stations = [station["coordinates"] for station in help_stations_with_phone]

phone = []


In [61]:
import requests
import folium
from folium import Circle, PolyLine

# --- User and Help Station Coordinates ---
user_location = (30.3245, 78.0430)

# --- Initialize phone list ---
phone = []

# --- Function to get distance using OSRM ---
def get_osrm_distance(origin, destination):
    url = f"http://router.project-osrm.org/route/v1/foot/{origin[1]},{origin[0]};{destination[1]},{destination[0]}"
    params = {"overview": "full", "geometries": "geojson"}
    response = requests.get(url, params=params)
    data = response.json()
    if 'routes' in data and len(data['routes']) > 0:
        distance_km = data['routes'][0]['distance'] / 1000
        coords = [(coord[1], coord[0]) for coord in data['routes'][0]['geometry']['coordinates']]
        return distance_km, coords
    else:
        return float('inf'), []

# --- Get distances from user to each help station ---
distances = []
for station in help_stations:
    dist_km, _ = get_osrm_distance(user_location, station)
    distances.append(dist_km)

# --- Get 3 Nearest Stations ---
nearest = sorted(zip(help_stations, distances), key=lambda x: x[1])[:3]

# --- Extract Phone Numbers for the Nearest Stations ---
for station, _ in nearest:
    for station_info in help_stations_with_phone:
        if station_info["coordinates"] == station:
            phone.append(station_info["phone_number"])

# --- Print distances and phone numbers of the 3 nearest help stations ---
print("\n📍 3 Nearest Help Stations (by walking distance):")
for i, (station, dist) in enumerate(nearest):
    print(f"{i+1}. Location: {station}, Distance: {dist:.2f} km, Phone: {phone[i]}")

# --- Create Map ---
m = folium.Map(location=user_location, zoom_start=15)

# User location
folium.Marker(user_location, tooltip="User", icon=folium.Icon(color='blue')).add_to(m)
Circle(user_location, radius=1500, color="blue", fill=False).add_to(m)

# All help stations (gray)
for station in help_stations:
    folium.Marker(station, tooltip="Help Station", icon=folium.Icon(color="gray")).add_to(m)

# Nearest stations (green)
for i, (station, dist) in enumerate(nearest):
    folium.Marker(
        station,
        tooltip=f"Help Station {i+1} ({dist:.2f} km), Phone: {phone[i]}",
        icon=folium.Icon(color="green")
    ).add_to(m)

    # Draw route from user to station
    _, coords = get_osrm_distance(user_location, station)
    PolyLine(coords, color='green', weight=4, opacity=0.7).add_to(m)

# Save map
m.save("live_osrm_nearest_help_stations_with_phone.html")
print("✅ Map saved as 'live_osrm_nearest_help_stations_with_phone.html'")
m



📍 3 Nearest Help Stations (by walking distance):
1. Location: (30.3196005, 78.0422984), Distance: 0.59 km, Phone: +917617641909
2. Location: (30.3257766, 78.0428731), Distance: 0.64 km, Phone: +917617641909
3. Location: (30.3184253, 78.0424823), Distance: 0.86 km, Phone: +919389969916
✅ Map saved as 'live_osrm_nearest_help_stations_with_phone.html'


In [62]:
print(phone)

['+917617641909', '+917617641909', '+919389969916']


In [63]:
from twilio.rest import Client

# Twilio credentials
account_sid = 'AC4c024fc964c6236c3b439abd84d49329'  # Replace with your Twilio Account SID
auth_token = 'e5d445e9dc0f544efe4ca61a8fb76c09'    # Replace with your Twilio Auth Token

# Twilio phone number
from_phone = '+19137330629'  # Replace with your Twilio number
message = "I love you :)"  # Message details


# Set up Twilio client
client = Client(account_sid, auth_token)

# Send SMS to each phone number in the list
for to_phone in phone:
    try:
        # Send SMS
        message_sent = client.messages.create(
            body=message,
            from_=from_phone,
            to=to_phone
        )
        print(f"Message sent to {to_phone}: {message_sent.sid}")
    except Exception as e:
        print(f"Failed to send message to {to_phone}: {e}")


Message sent to +917617641909: SM1a581f783c1bbd73196d0c88dc30bcea
Message sent to +917617641909: SM94a2728602e8e2371eb6c45c920ff2a0
Message sent to +919389969916: SM1ecbee189abbd8ce136520c16ee50639
